# Change in Dataset Construction

This notebook aims to change the way the dataset is built

In [77]:
import os
from pathlib import Path

try:
    os.chdir(PROJECT_PATH)
except NameError:
    NOTEBOOK_PATH = Path(os.getcwd())
    os.chdir("..")
    PROJECT_PATH = Path(os.getcwd())
SRC_PATH = PROJECT_PATH / "src"
DATA_PATH = PROJECT_PATH / "data"

In [179]:
import pandas as pd
import yfinance as yf
import datetime
import random
random.seed(10)
from tqdm import tqdm
import matplotlib.pyplot as plt
from src.utils.db import DBInterface
from src.utils.common import PositionalEncoding
import torch
from torch.nn import ModuleDict

from torch_geometric.nn import GATConv
import torch.nn.functional as F

from torch.optim import Adam, lr_scheduler


## Prepare Data Function

In [399]:
TARGETS = [
    "ITUB4.SA",
    "BBDC4.SA",
    "BBAS3.SA",
    "SANB11.SA",
    "BPAC11.SA",
    "PETR4.SA"
           ]

COMP_IND = ["P/L", "PL/ATIVOS", "M. EBIT", "ROA", "CAGR LUCROS 5 ANOS"]
TARGETS_LOW = [col.lower().replace(".sa", "") for col in TARGETS]

In [400]:
def df_prep(db, targets):
    df = db.read_sql(query="SELECT * FROM stocks")

    df_symbol = df[lambda f: f["symbol"].isin(targets)]

    assert not df_symbol.empty, "The dataframe is empty"

    df_pivot = df_symbol.pivot(
        index="quote_date",
        columns="symbol",
        values=["close"],
    )
    df_pivot.columns = [
        stock.lower().replace(".sa", "") + "_price" for _, stock in df_pivot.columns.values
    ]    
    # focus only on close price for now
    df_close = df_pivot.filter(like="price")
    
    # col_close_to_prices = [col.replace("close", "price") for col in df_close.columns]
    
    # df_price = df_close.copy()
    # df_price.columns =col_close_to_prices
    
    return df_close

In [401]:
db = DBInterface()
df = df_prep(db, targets = TARGETS).sort_index(ascending=False)

In [403]:
df_fund = pd.read_csv(DATA_PATH / "company_indicators_top5_banks.csv")

In [404]:
df_ind = df_fund.loc[lambda f: f["stock"].isin(TARGETS_LOW) & f["indicators"].isin(COMP_IND)]
df_ind["quote_date"] = pd.to_datetime(df_ind["quote_date"])
df_ind["col"] = df_ind["stock"] + "_" + df_ind["indicators"].str.lower().str.replace(".", "").str.replace(" ", "-")
df_pivot = df_ind.pivot(
    index = "quote_date",
    columns = "col",
    values="valor"
)
df_pivot["year"] = df_pivot.index.year

/var/folders/xk/602tgmp162ngdgy2l3rwdpn00000gn/T/ipykernel_56234/392088152.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_ind["quote_date"] = pd.to_datetime(df_ind["quote_date"])
/var/folders/xk/602tgmp162ngdgy2l3rwdpn00000gn/T/ipykernel_56234/392088152.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_ind["col"] = df_ind["stock"] + "_" + df_ind["indicators"].str.lower().str.replace(".", "").str.replace(" ", "-")


In [405]:
df["year"] = df.index.year

res = df.merge(
    df_pivot,
    how="left",
    on="year",
).drop("year", axis=1)

res.set_index(df.index, inplace=True)

multi = pd.MultiIndex.from_tuples([tuple(col.split("_"))for col in res.columns])

res.columns = multi

res = res.reorder_levels([0, 1], axis=1).sort_index(axis=1)

In [406]:
res

bbas3                                             \
           cagr-lucros-5-anos  m-ebit   p/l pl/ativos      price     roa   
quote_date                                                                 
2023-10-06             0.2508  0.1616  4.27      0.08  48.849998  0.0157   
2023-10-05             0.2508  0.1616  4.27      0.08  46.939999  0.0157   
2023-10-04             0.2508  0.1616  4.27      0.08  46.799999  0.0157   
2023-10-03             0.2508  0.1616  4.27      0.08  46.450001  0.0157   
2023-10-02             0.2508  0.1616  4.27      0.08  47.049999  0.0157   
...                       ...     ...   ...       ...        ...     ...   
2000-01-07                NaN     NaN   NaN       NaN   1.028783     NaN   
2000-01-06                NaN     NaN   NaN       NaN   1.052463     NaN   
2000-01-05                NaN     NaN   NaN       NaN   1.037993     NaN   
2000-01-04                NaN     NaN   NaN       NaN   1.027468     NaN   
2000-01-03                NaN     NaN   NaN       NaN   1.078775     NaN   

                        bbdc4                          ...     itub4  \
           cagr-lucros-5-anos  m-ebit   p/l pl/ativos  ... pl/ativos   
quote_date                                             ...             
2023-10-06            -0.0096  0.0619  9.46       NaN  ...      0.07   
2023-10-05            -0.0096  0.0619  9.46       NaN  ...      0.07   
2023-10-04            -0.0096  0.0619  9.46       NaN  ...      0.07   
2023-10-03            -0.0096  0.0619  9.46       NaN  ...      0.07   
2023-10-02            -0.0096  0.0619  9.46       NaN  ...      0.07   
...                       ...     ...   ...       ...  ...       ...   
2000-01-07                NaN     NaN   NaN       NaN  ...       NaN   
2000-01-06                NaN     NaN   NaN       NaN  ...       NaN   
2000-01-05                NaN     NaN   NaN       NaN  ...       NaN   
2000-01-04                NaN     NaN   NaN       NaN  ...       NaN   
2000-01-03                NaN     NaN   NaN       NaN  ...       NaN   

                                  petr4             sanb11                 \
                price    roa      price cagr-lucros-5-anos  m-ebit    p/l   
quote_date                                                                  
2023-10-06  27.900000  0.013  33.509998             0.0229  0.0732  10.38   
2023-10-05  27.639999  0.013  32.730000             0.0229  0.0732  10.38   
2023-10-04  27.209999  0.013  32.619999             0.0229  0.0732  10.38   
2023-10-03  26.600000  0.013  33.970001             0.0229  0.0732  10.38   
2023-10-02  26.940001  0.013  34.119999             0.0229  0.0732  10.38   
...               ...    ...        ...                ...     ...    ...   
2000-01-07        NaN    NaN   1.410173                NaN     NaN    NaN   
2000-01-06        NaN    NaN   1.403764                NaN     NaN    NaN   
2000-01-05        NaN    NaN   1.408635                NaN     NaN    NaN   
2000-01-04        NaN    NaN   1.422993                NaN     NaN    NaN   
2000-01-03        NaN    NaN   1.506321                NaN     NaN    NaN   

                                         
           pl/ativos      price     roa  
quote_date                               
2023-10-06      0.11  27.190001  0.0098  
2023-10-05      0.11  26.680000  0.0098  
2023-10-04      0.11  26.059999  0.0098  
2023-10-03      0.11  25.700001  0.0098  
2023-10-02      0.11  25.920000  0.0098  
...              ...        ...     ...  
2000-01-07       NaN        NaN     NaN  
2000-01-06       NaN        NaN     NaN  
2000-01-05       NaN        NaN     NaN  
2000-01-04       NaN        NaN     NaN  
2000-01-03       NaN        NaN     NaN  

[5971 rows x 31 columns]

In [407]:
df_macro = db.read_sql(query="SELECT * FROM macro")
df_macro_sorted = df_macro.pivot(
    index="quote_date",
    columns="indicators",
    values="valor"
).sort_index(ascending=False).reset_index()
df_macro_sorted["quote_date"] = pd.to_datetime(df_macro_sorted["quote_date"])
df_macro_sorted

indicators,quote_date,Dolar,IPCA,PIB,Risco-Brasil,Selic Over
0,2023-10-06,NaN,NaN,NaN,-1.422,NaN
1,2023-10-05,NaN,NaN,NaN,0.957,NaN
2,2023-10-04,NaN,NaN,NaN,1.951,NaN
3,2023-10-03,NaN,NaN,NaN,2.500,NaN
4,2023-10-02,NaN,NaN,NaN,-0.498,NaN
...,...,...,...,...,...,...
8169,1953-05-01,-4.085,NaN,NaN,NaN,NaN
8170,1953-04-01,11.252,NaN,NaN,NaN,NaN
8171,1953-03-01,0.000,NaN,NaN,NaN,NaN
8172,1953-02-01,100.165,NaN,NaN,NaN,NaN


In [408]:
df_diff = res.copy()
df_diff["diff"] = -(df_diff.reset_index()["quote_date"].shift(1) - df_diff.index).dt.days.fillna(0).values

In [409]:
df_macro = (
    pd.DataFrame(df_diff.reset_index()["quote_date"])
    .merge(df_macro_sorted, 
           on="quote_date", 
           how="left")
    .fillna(method="backfill")
    .fillna(0)
    .set_index("quote_date")
)

## Dataset Preparation

In [410]:
REMAP = {company: index + 1 for index, company in enumerate(df.columns)}

In [411]:
# Dimension of the temporal part of Positional Encoding
PE_T = 10

# Price history
HISTORY = 14

# Number of days to test
TEST_DAYS = 360

# Forecast days
HORIZON_FORECAST = 1

In [412]:
def add_time_component(df: pd.DataFrame, time: int, history: int, horizon: int) -> pd.DataFrame:
    """Based on the price database, 
    add the time component positional encoding 

    :param df: the dataframe in the window
    :type df: pd.DataFrame
    """
    
    # Filter the dataframe
    df_range = df_diff[time-horizon:time+history]

    # get prices
    df_range_prices = df_range.xs("price", axis=1, level=1,drop_level=True)

    # Keep symbol where there is all predicted prices AND at least one history price
    exist_price_history = (~df_range_prices.isna().all()) 
    exist_predicted_price = (~df_range_prices.isna().any())
    keep = exist_price_history & exist_predicted_price
    keep = pd.concat([keep,pd.Series({"diff": True})], axis=0)

    df_range = df_range[keep.index[keep]]

    # Companies
    companies = list(set(df_range.columns.get_level_values(0)) - {"diff"})

    # Define y
    y_list = df_range_prices.iloc[:horizon][companies].round(5).T.values.tolist()
        
    # cumulated the time component
    df_range.iloc[:horizon, -1] = 0
    df_order = df_range.assign(order=lambda x: x['diff'].cumsum())
    
    return df_order, y_list, companies

In [413]:
# extract prices history, futures and companies name
df_prices, y, companies = add_time_component(df=df_diff,
                    time=10,
                    history=HISTORY,
                    horizon=HORIZON_FORECAST)

In [414]:
pe = PositionalEncoding(PE_T)
N = df.shape[0]

data = {
    "train": {},
    "test": {},
    "pred": {},
    "macro": {}
}

for t in tqdm(range(HORIZON_FORECAST, N - HISTORY)):
    
    # extract prices history, futures and companies name
    df_prices, y, companies = add_time_component(df=df_diff,
                       time=t,
                       history=HISTORY,
                       horizon=HORIZON_FORECAST)
    
    
    data["pred"][t] = dict(zip(companies, y))
    
    macro_features = torch.tensor(df_macro.iloc[t].values, dtype=torch.float)
    
    if torch.isnan(macro_features).any():
            print("NaN Detected In Macro")
    
    data["macro"][t] =macro_features
    
    d_t = {}
    
    for col in companies:
        
        # get company and order component
        df_col = df_prices.loc[:, pd.IndexSlice[[col, 'order'], :]].dropna(subset = [(col, "price")]).fillna(0)
        
        # tensor
        tensor_col = torch.tensor(df_col.values, dtype=torch.float)  
        prices = tensor_col[HORIZON_FORECAST:,:-1]
        pos = tensor_col[HORIZON_FORECAST:,-1].unsqueeze(-1)
        
        # encode the position
        pos_enc = pe(pos)
        
        features = torch.concat((prices, pos_enc), dim = 1)
        
        if torch.isnan(features).any():
            print("NaN Detected In Features")

        # add features in a list to pad later
        d_t[col] = features
        
    if len(d_t) == 0:
        continue
        
    if t < TEST_DAYS:
        data["test"][t] = d_t
    else:
        data["train"][t] = d_t

100%|██████████| 5956/5956 [01:46<00:00, 55.77it/s] 


## Model

In [415]:
MACRO_SIZE = df_macro.shape[1]

In [416]:
def mape_loss(x, y):
    # Ensure no division by zero
    epsilon = 1e-6
    loss = torch.abs(100 * (x - y) / (y + epsilon))
    return torch.mean(loss)

In [417]:
d_size = {}
for comp in TARGETS_LOW:
    size = df_diff.loc[:, (comp, slice(None))].shape[1]
    d_size[comp] = size
    print(f"{comp} has {size-1} macro indicators")

itub4 has 5 macro indicators
bbdc4 has 5 macro indicators
bbas3 has 5 macro indicators
sanb11 has 5 macro indicators
bpac11 has 5 macro indicators
petr4 has 0 macro indicators


In [418]:
class CompanyExtractor(torch.nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(CompanyExtractor, self).__init__()
        self.lstm = torch.nn.RNN(input_size=input_size, hidden_size=hidden_size, num_layers=1)

    def forward(self, x):
        _, hn = self.lstm(x)
        return hn[0]
    
lstm_models = ModuleDict({comp: CompanyExtractor(size + PE_T, OUT_LSTM_SIZE, HORIZON_FORECAST) for comp, size in d_size.items()})
mlp_heads = ModuleDict({comp: torch.nn.Linear(OUT_GNN_SIZE + MACRO_SIZE, HORIZON_FORECAST) for comp in TARGETS_LOW})

In [419]:
class MyGNN(torch.nn.Module):
    def __init__(
        self,
        in_channels,
        out_channels,
    ):
        super(MyGNN, self).__init__()
        self.gat_conv = GATConv(in_channels=in_channels, out_channels=out_channels)
        
    def forward(self, data):
        lstm_tensor = data
        nbr_nodes = lstm_tensor.shape[0]
        edge_index = torch.combinations(torch.arange(nbr_nodes)).t()
        
        x = self.gat_conv(lstm_tensor, edge_index)
        x = F.relu(x)
        
        return x
    
my_gnn = MyGNN(in_channels = OUT_LSTM_SIZE,out_channels = OUT_GNN_SIZE)

In [420]:
def run_lstm_separatly(lstm_dict: torch.nn.ModuleDict, data_t: dict) -> torch.tensor:
    
    out_lstm = []
    comps = []
        
    # Run each LSTM separatly and aggregate the result in the same matrix
    for comp, tensor in data_t.items(): 
        tensor = tensor.unsqueeze(1)
        
        out_comp_lstm = lstm_dict[comp](tensor)
                        
        # Getting the last output from LSTM
        out_lstm.append(out_comp_lstm.squeeze(0))
        comps.append(comp)
                            
    # Concatenate the outputs 
    features_extracted = torch.stack(out_lstm, dim=0).squeeze(1)  
    
    return features_extracted, comps

def run_mlp_heads_separatly(mlp_heads: torch.nn.ModuleDict, features_encoded: torch.tensor, comps:list, pred_t: dict, macro: torch.tensor)-> torch.tensor:
    
    # Run each MLP separatly
    price_outputs_time_t = []
    pred_output_time_t = []
    
    for k, comp in enumerate(comps):
        out_gnn_comp_i = features_encoded[k]
        
        gnn_with_macro = torch.concatenate([out_gnn_comp_i, macro])
        
        price_comp_i = mlp_heads[comp](gnn_with_macro)
        price_outputs_time_t.append(price_comp_i)
        pred_output_time_t.append(pred_t[comp])
        
    # Concatenate the outputs frm the LSTM
    pred = torch.stack(price_outputs_time_t, dim=0)
    
    # Prepare ground truth from d_pred for the current timestep
    true = torch.tensor(pred_output_time_t).reshape_as(pred).float()
    
    return pred, true
    

In [423]:
HYPERPARAM = {
    out_lstm_size: 5,
    out_gnn_size: 5,
    criterion: "MSE",
    optimizer: "Adam",
    lr: 0.01,
    scheduler: "ReduceLROnPlateau",
    patience: 8,
    factor: 0.8,
    epochs = 300, 
}

RUN_CONFIG = {
    PE_T: 10,
    HISTORY: 14, 
    TEST_DAYS: 360,
    HORIZON_FORECAST: 1,
}

TEST_CONFIG = {
    use_gnn: True,
    var_macro: True,
    var_fund: True,
    stocks: "Banks" # or random 
}

NameError: name 'out_lstm_size' is not defined

In [421]:
# Define a loss function and optimizer
criterion = torch.nn.MSELoss()  # Mean Squared Error Loss for regression tasks
optimizer = Adam(list(lstm_models.parameters()) + list(my_gnn.parameters())+ list(mlp_heads.parameters()), lr=0.01)

# Learning rate scheduler
scheduler = lr_scheduler.ReduceLROnPlateau(optimizer, 'min', patience=8, factor=0.8, verbose=True)

# Training loop
num_epochs = 300

use_GNN = False

results_loss = {"train": [], "test": []}

for epoch in range(num_epochs):
    total_train_loss = 0.0
    total_test_loss = 0.0
    train_timesteps = list(data["train"].keys())[:500]
    test_timesteps = list(data["test"].keys())[:10]
    random.shuffle(train_timesteps)
    
    for timestep in tqdm(train_timesteps):
        data_t = data["train"][timestep]
        pred_t = data["pred"][timestep]
        macro = data["macro"][timestep]

        optimizer.zero_grad()
        
        # PHASE 1: LSTM EXTRACTION
        features_extracted, comps = run_lstm_separatly(lstm_models, data_t)
        
        # PHASE 2: GNN EXTRACTION
        if use_GNN:
            features_encoded = my_gnn(features_extracted)
        else:
            features_encoded = features_extracted
        
        # PHASE 3: MLP HEAD EXTRACTION
        pred, true = run_mlp_heads_separatly(mlp_heads, features_encoded, comps, pred_t, macro)
            
        # Compute the loss
        loss = mape_loss(pred, true)
        total_train_loss += loss.item()
         
        # Backward pass and optimization
        loss.backward()
        optimizer.step() 
        
    avg_train_loss = total_train_loss/len(train_timesteps)
    
    my_gnn.eval()  # Set the model to evaluation mode
    with torch.no_grad():
        for timestep in tqdm(test_timesteps):
            test_t = data["test"][timestep]
            pred_t = data["pred"][timestep]
            
            # PHASE 1: LSTM EXTRACTION
            features_extracted, comps = run_lstm_separatly(lstm_models, test_t)
            
            # PHASE 2: GNN EXTRACTION
            if use_GNN:
                features_encoded = my_gnn(features_extracted)
            else:
                features_encoded = features_extracted
            
            # PHASE 3: MLP HEAD EXTRACTION
            pred, true = run_mlp_heads_separatly(mlp_heads, features_encoded, comps, pred_t, macro)
            
                
            # Compute the loss
            loss = mape_loss(pred, true)
            total_test_loss += loss.item()
            
        avg_test_loss = total_test_loss/len(test_timesteps)
    
    results_loss["train"].append(avg_train_loss)
    results_loss["test"].append(avg_test_loss)
    # Update the learning rate
    scheduler.step(avg_test_loss)
    display(f"Epoch [{epoch+1}/{num_epochs}], Train Loss: {avg_train_loss:.4f}, Test Loss: {avg_test_loss:.4f}")

display("Training Finished!")

100%|██████████| 10/10 [00:00<00:00, 599.20it/s]


'Epoch [1/300], Train Loss: 45.4390, Test Loss: 42.2477'

100%|██████████| 10/10 [00:00<00:00, 573.49it/s]


'Epoch [2/300], Train Loss: 16.6154, Test Loss: 34.3257'

100%|██████████| 10/10 [00:00<00:00, 552.06it/s]


...

'Epoch [5/300], Train Loss: 13.4525, Test Loss: 33.0614'

100%|██████████| 10/10 [00:00<00:00, 562.91it/s]


'Epoch [6/300], Train Loss: 13.1311, Test Loss: 37.8653'

 32%|███▏      | 158/500 [00:01<00:02, 116.60it/s]


KeyboardInterrupt: 

In [274]:
NAIVE_COLS = [col + "_naive" for col in df.columns]
df_naive = df.copy().iloc[:360]

In [288]:
res = []
df_naive = df.copy().iloc[:360]
BASE_COLS = df_naive.columns
for k in range(1, HORIZON_FORECAST+1):
    NAIVE_COLS = [col + "_naive_" + str(k) for col in BASE_COLS]
    # print(NAIVE_COLS)
    # display(df_naive[BASE_COLS].shift(-k))
    df_naive[NAIVE_COLS] = df_naive[BASE_COLS].shift(-k)
    res_k = []
    for col in df.columns:
        x = df_naive[col]
        y = df_naive[col + "_naive_" + str(k)]
        res_k.append((abs(100*(x - y) / (y))).mean())
    res.append(res_k)

In [293]:
for e_k in res:
    print(np.mean(e_k))

1.3863443549317693
2.019333779733331
2.5116544197903066
2.951522337770069
3.3507373065127455


In [202]:
col = "close_b3sa3"
x = df_naive[col]
y = df_naive[col + "_naive"]
(abs(100*(x - y) / (y))).mean()

1.9520859410384888

In [185]:
loss = torch.abs(100 * (x - y) / (y + epsilon))
    return torch.mean(loss)

Index(['close_b3sa3', 'close_bbdc4', 'close_bees3'], dtype='object')